In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
pip install pypdf -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install docx2txt -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install wikipedia -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data


# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data
  

In [6]:
def insert_or_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings
    
    embeddings = OpenAIEmbeddings()
    
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name in pinecone.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creating index {index_name} and embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store
    

In [7]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.run(q)
    return answer

In [8]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

/Users/yashviradia/anaconda3/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Creating index askadocument and embeddings ...

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'Content-Length': '140', 'date': 'Sat, 16 Sep 2023 09:53:33 GMT', 'x-envoy-upstream-service-time': '3', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Capacity Reached. Starter Projects support a single index. Create a new project to add more. Your Starter Project remains free post-upgrade.


In [9]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deleting all indexes ... ')
        for index in indexes:
            pinecone.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('Ok')
    

In [10]:
data = load_from_wikipedia('GPT-4', 'de')
print(data[0].page_content)

OpenAI LP ist ein US-amerikanisches Unternehmen, das sich mit der Erforschung von künstlicher Intelligenz (KI, englisch Artificial Intelligence, AI) beschäftigt. Die gewinnorientierte Tochtergesellschaft OpenAI LP wird dabei durch das Non-Profit-Mutterunternehmen OpenAI Inc. kontrolliert.
Zentrale Geldgeber der Organisation sind der Unternehmer Elon Musk sowie das Unternehmen Microsoft. Stand 2015 war das Ziel von OpenAI, künstliche Intelligenz auf Open-Source-Basis auf eine Art und Weise zu entwickeln und zu vermarkten, dass sie der Gesellschaft Vorteile bringt und nicht schadet. Die Organisation wollte eine „freie Zusammenarbeit“ mit anderen Institutionen und Forschern, indem sie ihre Patente und Forschungsergebnisse für die Öffentlichkeit zugänglich mache. Die Firma ist mit über 1 Milliarde US-Dollar von Spenden finanziert.OpenAI beschäftigt sich nach eigenen Angaben mit der Frage der „existenziellen Bedrohung durch künstliche Intelligenz“ – also dem möglichen Übertreffen und Ersetz

In [11]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings ...

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'Content-Length': '140', 'date': 'Sat, 16 Sep 2023 10:02:22 GMT', 'x-envoy-upstream-service-time': '2', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Capacity Reached. Starter Projects support a single index. Create a new project to add more. Your Starter Project remains free post-upgrade.


In [12]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings ...

NameError: name 'chunks' is not defined

In [13]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks


In [14]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)


Index askadocument already exists. Loading embeddings ... Ok


In [15]:
q = 'What is the whole document about?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

I'm sorry, but I don't have access to the document you are referring to. Without any specific information, I cannot determine what the whole document is about.


In [17]:
data = load_document('files/SikaAnchor-300-en-MY-(06-2019)-1-1.pdf')


Loading files/SikaAnchor-300-en-MY-(06-2019)-1-1.pdf


In [ ]:
import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break
    
    answer = ask_and_get_answer(vector_store, q)
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

    

Write Quit or Exit to quit.
Question #1: What is context of this document?

Answer: I'm sorry, but I don't have any specific document in front of me to provide its context. Could you please provide more information or specify which document you are referring to?

 -------------------------------------------------- 

